# IDS_balance project: integration of `Sbrollini23` dataset

This notebook integrates the metadata file and all data files of a dataset into a single dataframe and a single parquet file.

`Sbrollini23` dataset: pandas DataFrame with 360000 rows × 35 columns with 41 MB in memory and 25 MB parquet file.

> Sbrollini, Agnese; Agostini, Valentina; Cavallini, Chanda; Burattini, Laura; Knaflitz, Marco (2023) Data for: Postural data from Stargardt's syndrome patients. Mendeley Data, V2, doi: 10.17632/5rz9j8t6p4.2
> V. Agostini, A. Sbrollini, C. Cavallini, A. Busso, G. Pignata, M. Knaflitz, The role of central vision in posture: Postural sway adaptations in Stargardt patients, Gait and Posture (2016). doi:10.1016/j.gaitpost.2015.10.003.

## Setup

In [1]:
import sys, os, datetime, glob
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.io as sio

print(f'Python {sys.version} on {sys.platform}',
      f' numpy {np.__version__}', f' pandas {pd.__version__}',
      datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"), sep='\n')

Python 3.12.7 | packaged by conda-forge | (main, Oct  4 2024, 16:05:46) [GCC 13.3.0] on linux
 numpy 2.2.1
 pandas 2.2.3
28/12/2024 00:32:04


## Dataset location

In [2]:
dataset_name = 'Sbrollini23'
metadata_fname = 'SubjectsData.xlsx'
path2 = Path().resolve().parents[0] / 'datasets' / dataset_name / 'data'
if os.path.isfile(path2 / metadata_fname):
    print(f'Dataset location: {path2}')
else:
    print('Dataset not found.')

Dataset location: /home/marcos/adrive/Python/projects/IDS_balance/datasets/Sbrollini23/data


## Metadata

In [3]:
metadata = pd.read_excel(path2 / metadata_fname, header=None, skiprows=4, na_values='-')
metadata.columns = ['n°', 'sex', 'age (years)', 'weight (kg)', 'height (cm)', "toes' distance (cm)",
                    'toe-heel distance (cm)', 'VADSR', 'VADSL', 'VAMARR', 'VAMARL', 'VALMARR', 'VALMARL',
                    'FSL2R (%)', 'FSL2L (%)', 'FSL4R (%)', 'FSL4L (%)', 'AFD (years)', 'date', 'order']
display(metadata)
print(f'Information from {len(metadata)} files successfully loaded (total of {len(pd.unique(metadata.iloc[:, 0]))} subjects).')

,n°,sex,age (years),weight (kg),height (cm),toes' distance (cm),toe-heel distance (cm),VADSR,VADSL,VAMARR,VAMARL,VALMARR,VALMARL,FSL2R (%),FSL2L (%),FSL4R (%),FSL4L (%),AFD (years),date,order
0,ST1,male,24,84,186,22.5,26.0,0.20,0.20,5.0,5.0,0.70,0.7,87.0,95.0,100.0,100.0,12.0,2014-03-19,O/O/M/M/M/O/M/C/O/O/M/C/C/C/C
1,ST2,male,37,85,180,21.0,25.0,0.20,0.20,5.0,5.0,0.70,0.7,100.0,35.0,100.0,94.0,16.0,2014-03-19,C/O/M/M/O/M/O/M/O/C/O/C/C/M/C
2,ST3,male,48,65,165,20.0,25.0,0.25,0.25,4.0,4.0,0.60,0.6,75.0,94.0,96.0,98.0,13.5,2014-04-02,O/O/O/C/C/C/O/M/M/C/C/M/M/M/O
3,ST4,male,39,69,183,23.0,27.0,0.15,0.10,6.7,10.0,0.82,1.0,97.0,97.0,100.0,100.0,10.0,2014-04-08,C/M/O/O/C/C/M/O/M/O/C/M/C/O/M
4,ST5,female,58,75,150,18.5,22.5,0.05,0.04,20.0,25.0,1.30,1.4,96.0,22.0,100.0,94.0,23.0,2014-05-05,C/O/O/C/M/M/M/O/O/C/C/M/M/C/O
5,ST6,female,23,58,152,18.5,22.5,0.05,0.04,20.0,25.0,1.30,1.4,17.0,45.0,51.0,85.0,13.0,2014-05-05,O/M/O/M/O/C/C/O/O/M/C/C/M/M/C
6,ST7,male,59,72,172,21.5,26.0,0.04,0.04,25.0,25.0,1.40,1.4,84.0,47.0,100.0,94.0,21.0,2014-05-05,M/O/C/C/O/O/O/M/C/M/C/M/C/M/O
7,ST8,male,25,67,180,23.0,26.0,0.06,0.08,16.7,12.5,1.20,1.1,85.0,22.0,99.0,59.0,13.0,2014-05-05,C/O/M/M/C/M/C/C/O/O/O/M/O/M/C
8,ST9,female,46,76,165,21.0,22.0,0.04,0.08,25.0,12.5,1.40,1.1,52.0,47.0,94.0,89.0,15.0,2014-05-06,O/O/M/C/M/C/O/C/M/C/C/O/M/O/M
9,ST10,female,17,55,170,20.5,21.0,0.10,0.10,10.0,10.0,1.00,1.0,92.0,71.0,100.0,95.0,15.0,2014-05-06,M/C/M/O/C/O/O/M/M/O/C/C/M/C/O


Information from 20 files successfully loaded (total of 20 subjects).


## Integration

### Expand metadata to contain conditions of all trials

In [4]:
order = metadata['order'].str.split('/', expand=True) #.add_prefix('T')
order.columns = range(1, 16)
order = order.add_prefix('Vision')
trials = pd.wide_to_long(pd.concat((metadata.iloc[:, 0], order), axis=1), 'Vision', 'n°', 'PostTest').reset_index()
metadata = pd.merge(metadata, trials, on='n°', how='inner')
metadata['Trial'] = metadata['n°'] + '_' + metadata['PostTest'].map(str)
cols = metadata.columns.to_list()
cols = [cols[0]] + cols[:-3:-1] + cols[1:-2]
metadata = metadata[cols]
display(metadata)

,n°,Trial,Vision,sex,age (years),weight (kg),height (cm),toes' distance (cm),toe-heel distance (cm),VADSR,...,VALMARR,VALMARL,FSL2R (%),FSL2L (%),FSL4R (%),FSL4L (%),AFD (years),date,order,PostTest
0,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,0.7,0.7,87.0,95.0,100.0,100.0,12.0,2014-03-19,O/O/M/M/M/O/M/C/O/O/M/C/C/C/C,1
1,ST1,ST1_2,O,male,24,84,186,22.5,26.0,0.2,...,0.7,0.7,87.0,95.0,100.0,100.0,12.0,2014-03-19,O/O/M/M/M/O/M/C/O/O/M/C/C/C/C,2
2,ST1,ST1_3,M,male,24,84,186,22.5,26.0,0.2,...,0.7,0.7,87.0,95.0,100.0,100.0,12.0,2014-03-19,O/O/M/M/M/O/M/C/O/O/M/C/C/C/C,3
3,ST1,ST1_4,M,male,24,84,186,22.5,26.0,0.2,...,0.7,0.7,87.0,95.0,100.0,100.0,12.0,2014-03-19,O/O/M/M/M/O/M/C/O/O/M/C/C/C/C,4
4,ST1,ST1_5,M,male,24,84,186,22.5,26.0,0.2,...,0.7,0.7,87.0,95.0,100.0,100.0,12.0,2014-03-19,O/O/M/M/M/O/M/C/O/O/M/C/C/C/C,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,CT10,CT10_11,O,male,44,96,177,22.0,26.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2014-06-19,O/M/C/C/M/M/C/M/C/M/O/O/O/O/C,11
296,CT10,CT10_12,O,male,44,96,177,22.0,26.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2014-06-19,O/M/C/C/M/M/C/M/C/M/O/O/O/O/C,12
297,CT10,CT10_13,O,male,44,96,177,22.0,26.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2014-06-19,O/M/C/C/M/M/C/M/C/M/O/O/O/O/C,13
298,CT10,CT10_14,O,male,44,96,177,22.0,26.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2014-06-19,O/M/C/C/M/M/C/M/C/M/O/O/O/O/C,14


In [5]:
metadata = metadata.astype('category', copy=True)
metadata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   n°                      300 non-null    category
 1   Trial                   300 non-null    category
 2   Vision                  300 non-null    category
 3   sex                     300 non-null    category
 4   age (years)             300 non-null    category
 5   weight (kg)             300 non-null    category
 6   height (cm)             300 non-null    category
 7   toes' distance (cm)     300 non-null    category
 8   toe-heel distance (cm)  300 non-null    category
 9   VADSR                   300 non-null    category
 10  VADSL                   300 non-null    category
 11  VAMARR                  300 non-null    category
 12  VAMARL                  300 non-null    category
 13  VALMARR                 300 non-null    category
 14  VALMARL                 30

### Merge metadata and data files individually and then concatenate all

In [6]:
def merge_meta_data(metadata, trial):
    # Merge metadata and data files
    fname, test_n = trial.split('_')
    mat = sio.loadmat(path2.as_posix() + os.sep + fname + '.mat')
    sig = mat['Sig']
    test_names = list(sig.dtype.names)
    grf_names = ['time'] + list(sig[0, 0][test_names[int(test_n)-1]].dtype.names)[1:-1]
    dados = np.empty(shape=(sig[0, 0][test_names[int(test_n)-1]][grf_names[1]][0, 0].shape[0], len(grf_names)))
    dados[:, 0] = np.linspace(0, 60, 1200, endpoint=False)  # frequency = 20 Hz 
    for c, col in enumerate(grf_names[1:]):
        dados[:, c+1] = sig[0, 0][test_names[int(test_n)-1]][col][0, 0][:, 0]
        
    data = pd.DataFrame(data=dados, columns=grf_names)
    data['Trial'] = trial
    return pd.merge(metadata.query('Trial == @trial'), data, how='inner', on='Trial')

In [7]:
df_all = [merge_meta_data(metadata, trial) for trial in tqdm(metadata['Trial'])]
df_all = pd.concat(df_all, ignore_index=True)
df_all = df_all.astype({'Trial': 'category'})
df_all

100%|███████████████████████████████████████████████████████| 300/300 [00:58<00:00,  5.10it/s]


,n°,Trial,Vision,sex,age (years),weight (kg),height (cm),toes' distance (cm),toe-heel distance (cm),VADSR,...,Fy,Fz,Mx,My,Mz,Mx1,My1,COPx,COPy,Tz
0,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.180542,837.179871,-1529.469299,23979.069519,4.299927,-1525.497375,23958.385437,-0.523772,0.723570,0.846375
1,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.133362,837.179382,-1452.293396,24076.908875,23.115540,-1449.359436,24059.973816,-0.616158,0.775891,20.615477
2,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.117676,836.967590,-1432.017517,24333.448792,22.924805,-1429.428650,24315.154846,-0.899261,0.760584,20.926312
3,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.086304,837.033142,-1526.179504,24736.277771,15.846252,-1524.280823,24713.051819,-1.343220,0.608661,15.220702
4,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.062805,836.961304,-1505.920410,24992.807007,9.236145,-1504.538696,24962.782593,-1.614929,0.593305,9.816245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359995,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.539490,999.968933,19964.976501,40039.506531,440.362549,19931.107727,40059.517883,0.448158,1.614932,520.165719
359996,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.617615,1000.104736,20043.545532,40430.383301,398.056030,20007.958008,40443.404175,0.091925,1.658116,475.311654
359997,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.609680,1000.382812,20274.938965,40674.981689,390.190125,20239.526001,40691.395752,-0.122555,1.853076,470.759998
359998,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.727112,999.891174,20603.297424,41200.825500,367.872620,20565.300964,41216.887756,-0.645933,2.157872,454.082954


In [8]:
df_all.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360000 entries, 0 to 359999
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   n°                      360000 non-null  category
 1   Trial                   360000 non-null  category
 2   Vision                  360000 non-null  category
 3   sex                     360000 non-null  category
 4   age (years)             360000 non-null  category
 5   weight (kg)             360000 non-null  category
 6   height (cm)             360000 non-null  category
 7   toes' distance (cm)     360000 non-null  category
 8   toe-heel distance (cm)  360000 non-null  category
 9   VADSR                   360000 non-null  category
 10  VADSL                   360000 non-null  category
 11  VAMARR                  360000 non-null  category
 12  VAMARL                  360000 non-null  category
 13  VALMARR                 360000 non-null  category
 14  VALM

## Save data to file and test it

Use engine 'fastparquet' to preserve category data types; see: https://www.practicaldatascience.org/html/parquet.html

In [9]:
df_all.to_parquet(path2 / f'{dataset_name.lower()}.parquet', engine='fastparquet', index=False)

In [10]:
df_all2 = pd.read_parquet(path2 / f'{dataset_name.lower()}.parquet', engine='fastparquet')
df_all2

,n°,Trial,Vision,sex,age (years),weight (kg),height (cm),toes' distance (cm),toe-heel distance (cm),VADSR,...,Fy,Fz,Mx,My,Mz,Mx1,My1,COPx,COPy,Tz
0,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.180542,837.179871,-1529.469299,23979.069519,4.299927,-1525.497375,23958.385437,-0.523772,0.723570,0.846375
1,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.133362,837.179382,-1452.293396,24076.908875,23.115540,-1449.359436,24059.973816,-0.616158,0.775891,20.615477
2,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.117676,836.967590,-1432.017517,24333.448792,22.924805,-1429.428650,24315.154846,-0.899261,0.760584,20.926312
3,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.086304,837.033142,-1526.179504,24736.277771,15.846252,-1524.280823,24713.051819,-1.343220,0.608661,15.220702
4,ST1,ST1_1,O,male,24,84,186,22.5,26.0,0.2,...,-0.062805,836.961304,-1505.920410,24992.807007,9.236145,-1504.538696,24962.782593,-1.614929,0.593305,9.816245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359995,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.539490,999.968933,19964.976501,40039.506531,440.362549,19931.107727,40059.517883,0.448158,1.614932,520.165719
359996,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.617615,1000.104736,20043.545532,40430.383301,398.056030,20007.958008,40443.404175,0.091925,1.658116,475.311654
359997,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.609680,1000.382812,20274.938965,40674.981689,390.190125,20239.526001,40691.395752,-0.122555,1.853076,470.759998
359998,CT10,CT10_15,C,male,44,96,177,22.0,26.0,1.0,...,1.727112,999.891174,20603.297424,41200.825500,367.872620,20565.300964,41216.887756,-0.645933,2.157872,454.082954


In [11]:
df_all2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360000 entries, 0 to 359999
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   n°                      360000 non-null  category
 1   Trial                   360000 non-null  category
 2   Vision                  360000 non-null  category
 3   sex                     360000 non-null  category
 4   age (years)             360000 non-null  category
 5   weight (kg)             360000 non-null  category
 6   height (cm)             360000 non-null  category
 7   toes' distance (cm)     360000 non-null  category
 8   toe-heel distance (cm)  360000 non-null  category
 9   VADSR                   360000 non-null  category
 10  VADSL                   360000 non-null  category
 11  VAMARR                  360000 non-null  category
 12  VAMARL                  360000 non-null  category
 13  VALMARR                 360000 non-null  category
 14  VALM